Nearest neighbours is one of the simplest predictive models there is. It makes no mathematical assumptions, and it doesn't required any sort of heavy machinery. The only thing it requires are:
- Some notion of distance
- An assumption that points that are close to one another are similar

In the general situation, we have some data points and we have a corresponding set of labels. The labels could be **True** and **False  **, indicating whether each input satisfies some conditions like "Is spam?" or "is poisonous?" or "would be enjoyable to watch?" they could be categories, like movie ratings (G, PG, PG-13, R, NC-17). Or they could be favorite programming languages.
 In our case, the data points will be vectors, which means that we can use the distance function.
 Let's say we've picked a number **k** like 3 or 5.  Then when we want to classify some new data point, we find the **k** nearest labeles points and let them vote on the new output.

 To do this, we'll need a function that counts votes. One possibility is:

In [3]:
from collections import Counter
def raw_majority_vote(labels):
    votes = Counter(labels)
    winner, _ =votes.most_common(1)[0]
    return winner

print(raw_majority_vote([1,2,3,4,5,6,8,4,1,2,1,2,3]))
print(raw_majority_vote([1,2,1,2]))

1
1


But this doesn't do anything intelligent with ties. For example , imagine we're rating movies and the five nearest movies are rated G, G, PG, PG and R. Then G hase two votes and PG also has two votes. In that case, we have several options:
- Pick one of the winners at random.
- Weight the votes by distance and pick the weighted winner.
- Reduce **k** until we find a unique winner.

We'll implement the third:

In [4]:
def majority_vote(labels):
    ''' assumes that labels are ordered from nearest to farthest'''
    vote_couts = Counter(labels)
    winner , winner_count = vote_couts.most_common(1)[0]
    num_winners = len([count for count in vote_couts.values() if count == winner_count])

    if num_winners == 1:
        return winner # Unique winner, so return it
    else:
        return majority_vote(labels[:-1])
        # try again without the farthest

print(majority_vote([1,2,1,2,3,4,5,6,7,8,8]))

1


This approach is sure to work eventually, since in the worst case we go all the way down to just one label, at which point that one label wins.
With this function it's easy to create a classifier:

In [5]:
def vector_add(v,w):
    ''' add correspnding elements'''
    return [v_i + w_i for v_i, w_i in zip(v, w)]

def vector_subtract(v, w):
    ''' subtracts corresponding element'''
    return [v_i - w_i for v_i, w_i in zip(v, w) ]

def vector_sum(vectors):
    '''sum all corresponding elements'''
    result = vectors[0]               # Start with the first vector
    for vector in vectors[1:]:       # then loop over the others
        result = vector_add(result, vector) # and add them to the result
    return result

def scalar_multiply(c,v):
    ''' c is a number, v is a vector'''
    return [c * v_i for v_i in v]

def vector_mean(vectors):
    '''compute the vector whose ith element is the mean of the ith eleents of the input vectors'''
    n = len(vectors)
    return scalar_multiply(1/n , vector_sum(vectors))

def dot( v, w):
    ''' v_1 * w_1 + .....+ v_n * w_n'''
    return sum(v_i*w_i for v_i, w_i in zip(v,w))

def sum_of_squares(v):
    ''' v_1 * v_1 + .....+ v_n * v_n'''
    return dot(v,v)

import math

def magnitude(v):
    return math.sqrt(sum_of_squares(v)) # math.sqrt is square root function

def squared_distance(v,w):
    '''(v_1 - w_1)**2 + ...... + (v_n - w_n)**2'''
    return sum_of_squares(vector_subtract(v,w))

def distance( v,w):
    return math.sqrt(squared_distance(v,w))


In [12]:
def knn_classify(k , labeld_points, new_points):
    ''' each labeled point should be a pair (point, label)'''
    # order the labeled points from nearest to farthest
    by_distance = sorted(labeled_points, key = lambda point, _:distance(point, new_points))

    # Find the labls for the k closest
    k_nearest_labels = [label for _, label in by_distance[:k]]

    # and let them vote
    return majority_vote(k_nearest_labels)